In [ ]:
import win32com.client
import os
import numpy as np
import time
from os import getcwd
from os.path import join, dirname, exists
from math import pi, sqrt
from scipy.io import loadmat
import re
import matplotlib.pyplot as plt
# # path='D:/KangDH/Optislang_Motorcad/IPM_HDEV_2/MCAD/HDEV_Thesis2/DutyCycleData/'
# # ext_Duty_Cycle='OP1_temp_rise'
# # ext_Duty_Cycle_full=join(path,ext_Duty_Cycle)+'.dat'
# # ext_Duty_Cycle_full

# # power_filename=ext_Duty_Cycle+'_power.csv'
# # power_filename
# # from MotorCAD_Methods import MotorCAD
mcApp = win32com.client.Dispatch("MotorCAD.AppAutomation")  # Launch Motor-CAD application

# # mc=MotorCAD()
# ext_Duty_Cycle='OP1_temp_rise'
# %reset_selective -f ^(?!mcApp).*$

In [325]:
def fun_Turn_byAmpT(i_AmpT,i_Line_Current_RMS):
    res = i_AmpT/i_Line_Current_RMS
    return res

In [ ]:
### Lab: peak performance
    i_Turns_Coil    = fun_Turn_byAmpT(i_AmpT_rms,i_Line_Current_RMS)*p_Parallel_Path   # Number of turns per coil
    mcApp.SetVariable('TurnsCalc_MotorLAB', i_Turns_Coil)            # Turns per coil
    mcApp.SetVariable("LabThermalCoupling", 0)                         # Coupling with Thermal
    mcApp.SetVariable("OpPointSpec_MotorLAB", 0)                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("SpeedDemand_MotorLAB", p_Speed_Peak_Array[0])                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("TorqueDemand_MotorLAB", OP1[2])                       # 0- Torque 4-Max temperature definition  
    mcApp.CalculateOperatingPoint_Lab()                             # Operating point calculation

    o_LabPeak_IPeak, OLabPeak_beta=fun_Ipk_beta_by_Trq()


In [ ]:
def fun_Ipk_beta_by_Trq():
    ex, ipk = mcApp.GetVariable("LabOpPoint_StatorCurrent_Line_Peak")    # Get shaft torque value
    ex, beta = mcApp.GetVariable("LabOpPoint_PhaseAdvance")    # Get shaft torque value
    return ipk, beta

# Lab fun_find_current & remake DutyCycle on OSL_Device

In [ ]:
OP1=['OP1_temp_rise',1700,1300]
OP2=['OP2_temp_rise',1700,900]
OP3=['OP3_temp_rise',4000,380]


In [284]:
OP_list=[]
    # mcApp.CalculateDutyCycle_Lab()
    
def fun_Find_Ipk_4Trq65C_mk_dat(ext_Duty_Cycle,i_Turns_Coil):
    mcApp.SetVariable('TurnsCalc_MotorLAB', i_Turns_Coil)            # Turns per coil
    mcApp.SetVariable("LabThermalCoupling", 0)                         # Coupling with Thermal
    mcApp.SetVariable("OpPointSpec_MotorLAB", 0)                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("SpeedDemand_MotorLAB", ext_Duty_Cycle[1])                       # 0- Torque 4-Max temperature definition
    mcApp.SetVariable("TorqueDemand_MotorLAB", ext_Duty_Cycle[2])                       # 0- Torque 4-Max temperature definition  
    mcApp.CalculateOperatingPoint_Lab()                             # Operating point calculation
    ex, ipk = mcApp.GetVariable("LabOpPoint_StatorCurrent_Line_Peak")    # Get shaft torque value
    ex, beta = mcApp.GetVariable("LabOpPoint_PhaseAdvance")    # Get shaft torque value
    before,after =fun_Save_Duty_Cycle_Change_I(ext_Duty_Cycle,ipk)
    return ipk, beta, after

# Op1_i,OP1_beta,Op1_after=fun_Find_Ipk_4Trq65C_mk_dat(OP1,11)
# Op2_i,OP2_beta,Op2_after=fun_Find_Ipk_4Trq65C_mk_dat(OP2,11)
# Op3_i,OP3_beta,Op3_after=fun_Find_Ipk_4Trq65C_mk_dat(OP3,11)



Make external DutyCycle

In [283]:
# cur.split()
# mcApp.SetArrayVariable("Duty_Cycle_Current_Start"           , 0, 600)             # Layer 1 Magnet_Segments

# ex, cur=mcApp.GetArrayVariable('Duty_Cycle_Current_Start',6 )

## SetArrayVariable loop

def fun_Save_Duty_Cycle_Change_I(ext_Duty_Cycle,Ipk):
    ## Load reference Duty Cycle with ext_Duty_Cycle_name
    ref_Duty_Cycle=join(dirname(dirname(OSL_PROJECT_DIR)), 'DutyCycleData',ext_Duty_Cycle[0])+'.dat'
    mcApp.loadDutyCycle(ref_Duty_Cycle)

    ## change current
    ex, Duty_Cycle_Num=mcApp.GetVariable('Duty_Cycle_Num_Periods')
    ex, Duty_Cycle_Current_before=mcApp.GetVariable('Duty_Cycle_Current_Start')
    for i in range(0,Duty_Cycle_Num):   
        mcApp.SetArrayVariable('Duty_Cycle_Current_Start',i,Ipk/sqrt(2))
    ex, Duty_Cycle_Current_After=mcApp.GetVariable('Duty_Cycle_Current_Start')
    
    ## Save Duty Cycle current    
    ext_Duty_Cycle_new=join(mot_file_dir,'DutyCycleData',ext_Duty_Cycle[0])+'_new.dat'
    mcApp.SaveDutyCycle(ext_Duty_Cycle_new)
    return Duty_Cycle_Current_before,Duty_Cycle_Current_After



# fun_Calc_Temp_ext_duty_rename

In [296]:
def fun_Calc_Temp_ext_duty_rename(ext_Duty_Cycle):
    ext_Duty_Cycle_new=join(mot_file_dir,'DutyCycleData',ext_Duty_Cycle[0])+'_new.dat'
    mcApp.LoadDutyCycle(ext_Duty_Cycle_new)
    mcApp.SetVariable('InitialTransientTemperatureOption',3)
    mcApp.CalculateDutyCycle_Lab()
    fun_Rename_Matfile_Lab_Duty(ext_Duty_Cycle)
    

Duty Cycle Coupled Transient 해석 결과로 나오는 mat파일&csv folder 이름 변경


In [295]:
def fun_Rename_Matfile_Lab_Duty(ext_Duty_Cycle):
    ex, motpath=mcApp.GetVariable("CurrentMotFilePath_MotorLAB")
    motpath=re.sub(".mot","",motpath)
    Lab_path=motpath+'/Lab/'
    os.chdir(Lab_path)
    # mat rename
    rename_matfile=ext_Duty_Cycle[0]+'_lab_result.mat'
    if os.path.exists('MotorLAB_drivecycledata.mat'):
        os.rename('MotorLAB_drivecycledata.mat',rename_matfile)
    rename_csvpath=ext_Duty_Cycle[0]+'_lab_result_csv'
    # csv rename
    if os.path.exists('MotorLAB_drivecycledata'):
        os.rename('MotorLAB_drivecycledata',rename_csvpath)
    
    # temp rise rename
    os.chdir(motpath+'/Thermal/')
    if os.path.exists('transient.txt'):
        os.rename('transient.txt',ext_Duty_Cycle[0]+'_transient.txt')

# fun_Rename_Matfile_Lab_Duty('OP1_temp_rise')

# ext_Duty_Cycle='OP1_temp_rise'

# MotorLAB_drivecycledata




fun_Load_Temp_Rise_2csvfile

In [ ]:


# ext_Duty_Cycle='OP1_temp_rise'

# def fun_Load_Temp_Rise_2csvfile(lab_transient_fullpath_w_filename):
#     Temp_filename=lab_transient_fullpath_w_filename+'_temp.csv'
#     power_filename=lab_transient_fullpath_w_filename+'_power.csv'
#     mcApp.ExportResults('Transient', power_filename)
#     mcApp.ExportResults('Transient', Temp_filename)
    
# # fun_Load_Temp_Rise_2csvfile(ext_Duty_Cycle)

# fun_load_matfile - mat 파일에서 온도에 해당하는 정보 뽑아오기

In [304]:
OP1[0]

'OP1_temp_rise'

In [306]:
def fun_load_matfile_by_OP(ext_Duty_Cycle):
        ## This mat file consisted of Ndarray when we are using loadmat function
    ex, motpath=mcApp.GetVariable("CurrentMotFilePath_MotorLAB")
    motpath=re.sub(".mot","",motpath)
    Lab_path=motpath+'/Lab/'
    os.chdir(Lab_path)
    name_matfile=ext_Duty_Cycle[0]+'_lab_result.mat'
    Mat_File_Data=loadmat(name_matfile)
    return Mat_File_Data

OP1_temp_rise_mat=fun_load_matfile_by_OP(OP1)
AG_Temp=OP1_temp_rise_mat['Airgap_Temp']
AG_Temp=AG_Temp.flatten().tolist()

fun_check_temp_rise_allcomponent

In [318]:
def fun_Check_Temp_Rise_allComponent(ext_Duty_Cycle):
    ## init
    init_final_temp=[]
    ## change name of mat file
    fun_rename_matfile_lab_duty(ext_Duty_Cycle[0])
    ## load mat file
    list_from_mat=fun_load_matfile(ext_Duty_Cycle[0])
    
    ## 
    mat_temp=[list_from_mat.get(key) for key in list_from_mat.keys() if 'Temp' in key]
    mat_temp_key=[key for key in list_from_mat.keys() if 'Temp' in key]

    ## check temp rise
    for i in range(len(mat_temp)):
        temp=mat_temp[i].ravel().tolist()
        init_final_temp.append((temp[0],temp[-1]))
        check_temp=max(max(init_final_temp))
    
    ## 
    max_temp_key=[key for key in mat_temp_key if max(list_from_mat.get(key))==check_temp]
    dic_init_final_temp=dict(zip(mat_temp_key,init_final_temp))
    
    return check_temp, max_temp_key, dic_init_final_temp

a, b,c =fun_Check_Temp_Rise_allComponent(OP1)

In [ ]:
## Dict to list (which is array)

mat_temp=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
mat_time=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Time' in key]
mat_temp_key=[key for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
mat_torque=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Shaft_Torque' in key]
mat_emtorque=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Electromagnetic_Torque' in key]
mat_current=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Stator_Current_Line_Peak' in key]
mat_voltage=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Voltage_Phase_Peak' in key]
mat_Terminal_Power=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Terminal_Power' in key]
mat_Perror=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Power_Error' in key]


In [ ]:
# OP1_temp_rise_temp_dic=dict(zip(OP1_temp_rise_temp_key,OP1_temp_rise_temp))
# key=OP1_temp_rise_temp_dic.keys()
# max(OP1_temp_rise_temp_dic.get('Magnet_Temp'))

# Trf file에서 온도 상승량 확인

# [Plot]온도시험 결과값 확인, 입력, 출력, 온도 - with list data (which is ndarray type)


In [309]:

from matplotlib import legend

fig=plt.figure(figsize=(10,20))
time=mat_time[0].ravel().tolist()
for i in range(len(mat_temp)):
    temp=mat_temp[i].ravel().tolist()
    label=mat_temp_key[i]
    plt.subplot(4,1,1)
    plt.annotate(label+str(round(max(temp),2))+'degC',xy=(time[-1],temp[-1]))
    plt.plot(time,temp,label=label)
    plt.title('Temp Rise')
    plt.xlabel('Time(sec)')
    plt.ylabel('Temperature(degC)')
    

plt.legend()
plt.subplot(4,1,2)
plt.plot(time,mat_torque[0].ravel().tolist(), label='Shaft_Torque')
plt.plot(time,mat_emtorque[0].ravel().tolist(),label='Electromagnetic_Torque')
plt.legend()

plt.subplot(4,1,3)
plt.plot(time,mat_current[0].ravel().tolist(),label='Stator_Current_Line_Peak',)
plt.ylim(750,800)
plt.legend()

plt.subplot(4,1,4)
plt.plot(time,mat_voltage[0].ravel().tolist(),label='Voltage_Phase_Peak',)
plt.legend()
# plt.plot(OP1_temp_rise_time[0],OP1_temp_rise_temp[0][1])
fig.savefig(join(ext_Duty_Cycle+'png'))

NameError: name 'mat_time' is not defined

<Figure size 1000x2000 with 0 Axes>

In [ ]:
init_temp=temp[0]
final_temp=temp[-1]

In [299]:
list_from_mat=fun_load_matfile_by_OP(OP[1])


NameError: name 'fun_load_matfile_by_OP' is not defined

In [ ]:
# mat_temp=[OP1_temp_rise_mat.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]

# def check_temp_rise_allcomponent(ext_Duty_Cycle):
#     ## init
#     init_final_temp=[]
    
#     ## load mat file
#     list_from_mat=fun_load_matfile(ext_Duty_Cycle)
#     ## check temp rise
#     for i in range(len(list_from_mat)):
#         temp=list_from_mat[i].ravel().tolist()
#         init_final_temp.append((temp[0],temp[-1]))
#         check_temp=max(max(init_final_temp))
#     return init_final_temp, check_temp

# init_final_temp, check_temp=check_temp_rise_allcomponent(mat_temp)


        
    

In [ ]:
# def fun_check_init_final_temp(ext_Duty_Cycle):
#     Mat_File_Data=fun_load_matfile(ext_Duty_Cycle)
#     mat_temp=[Mat_File_Data.get(key) for key in OP1_temp_rise_mat.keys() if 'Temp' in key]
#     mat_temp_key=[key for key in OP1_temp_rise_mat.keys() if 'Temp' in key]

#     Mat_File_temp_key=Mat_File_Data.keys()

#     max_temp=max(Mat_File_temp)
#     min_temp=min(Mat_File_temp)
#     return max_temp, min_temp

# fun_check_init_final_temp('OP1_temp_rise')

In [ ]:

# # Data for plotting
# OSL_DESIGN_NAME = 'Design0001'
# fig, ax = plt.subplots()
# ax.plot(RotorPosition, CoggingTorqueVW)
# screenname='CoggingTorqueVW'
# ax.set(xlabel='Rotor Position (Mdeg)', ylabel='CoggingTorqueVW [Nm]',
#        title='CoggingTorqueVW')
# ax.grid()
# wdir='Z:/Thesis/Motorcad_Optislang/IPM_HDEV_1/OPD\HDEV_Thesis1.opd/HDEV__Thesis1/'
# savedir=join(wdir, 'Design0001' + '/' +  '_Pic_' + screenname )
# fig.savefig(join(savedir+'png'))
# plt.show()
# data_array=[np.transpose(RotorPosition), np.transpose(CoggingTorqueVW)]
# io.savemat(join(savedir+'.mat'),{"CoggingTorqueVW":data_array})
# # OP1_temp_rise_mat.keys()

Save CSV filed Manually

In [ ]:


# def fun_load_temp_rise_2csvfile(lab_transient_fullpath_w_filename):
#     mcApp.ExportResults('Transient', power_filename)
#     Temp_filename=ext_Duty_Cycle+'_temp.csv'
#     power_filename=ext_Duty_Cycle+'_power.csv'

# def fun_load_temp_rise_from_MCgraph()
#     temp_rise=open(Temp_filename,'r')



OSL Path allocation

 File 
 $$\$$
    - ef.MOT
    - refDutyCycle.mat : OP1_Temprise_DutyCycle.mat

    
Python
$$\$$
Result
$$\$$
    Lab/dutycycledata.mat -> 
    - Lab/elecdata.mat        %efficiency map
    - CoggingTorqueData.mat   %Cogging 
    - ForceData.mat           %Force




In [269]:
# Dir
OSL_PROJECT_DIR = 'Z:/Thesis/Optislang_Motorcad/HDEV_Code2/OPD/HDEV_Pjt1.opd' # OSL
OSL_DESIGN_DIR = join(OSL_PROJECT_DIR, 'Sensitivity1')  #OSL
refdir = join(dirname(dirname(OSL_PROJECT_DIR)), 'MCAD')

#Name
OSL_DESIGN_NAME= 'Design001'
Design_Name     = "HDEV_Model2"   # Reference Motor-CAD design

# os.makedirs(OSL_PROJECT_DIR)
# os.makedirs(OSL_DESIGN_DIR)
# os.makedirs(refdir)

In [275]:

wdir = OSL_DESIGN_DIR
mot_file_new_path = join(wdir, Design_Name + '_' + OSL_DESIGN_NAME + ".mot") # Path to the new *.mot file
mot_file_dir = join(wdir, Design_Name + '_' + OSL_DESIGN_NAME)
# ref_Duty_Cycle=join(dirname(dirname(OSL_PROJECT_DIR)), 'DutyCycleData',ext_Duty_Cycle_name)+'.dat'
# os.makedirs(mot_file_dir)
os.makedirs(join(mot_file_dir,'DutyCycleData'))


# Test executing OSL



Build Lab Time :

1. Prework

In [ ]:
Op1_i,OP1_beta,Op1_after=fun_Find_Ipk_4Trq65C_mk_dat(OP1,11)
Op2_i,OP2_beta,Op2_after=fun_Find_Ipk_4Trq65C_mk_dat(OP2,11)
Op3_i,OP3_beta,Op3_after=fun_Find_Ipk_4Trq65C_mk_dat(OP3,11)


2. Temp Rise Duty Cycle Data Calc Time :

In [ ]:
fun_Calc_Temp_ext_duty_rename(OP1)

In [297]:
fun_Calc_Temp_ext_duty_rename(OP2)

In [298]:
fun_Calc_Temp_ext_duty_rename(OP3)

In [ ]:
o_OP1_max_temp,OP1_max_pos,OP1_temp_dic=fun_Check_Temp_Rise_allComponent(OP1)
o_OP2_max_temp,OP2_max_pos,OP2_temp_dic=fun_Check_Temp_Rise_allComponent(OP2)
o_OP3_max_temp,OP3_max_pos,OP3_temp_dic=fun_Check_Temp_Rise_allComponent(OP3)
print(o_OP1_max_temp,o_OP2_max_temp,o_OP3_max_temp,OP3_max_pos,OP2_max_pos)

OP3_temp_dic['Stator_Winding_Temp_Average'][1]

In [ ]:
def fun_OP_temp_contraints(ext_Duty_Cycle,i_Turns_Coil):
    Op_i,OP_beta,Op_after=fun_Find_Ipk_4Trq65C_mk_dat(ext_Duty_Cycle,i_Turns_Coil)
    fun_Calc_Temp_ext_duty_rename(ext_Duty_Cycle)
    o_max_temp,max_pos,temp_dic=fun_Check_Temp_Rise_allComponent(ext_Duty_Cycle)
    return o_max_temp,max_pos,temp_dic, Op_i,OP_beta,Op_after






커플드 해석
    OP1 - 6:36s
    OP2 - 11.39.7
    OP3 - 14:17.2
일방해석 - 고온,저온


Driving Duty Cycle Data Calc Time :
    M1

In [ ]:
# op1=[1700,1300]
# init_coil=11
# OP1_Duty_Cycle='OP1_temp_rise'
# ipk, beta = fun_Find_Ipk_4Trq65C_mk_dat(op1,init_coil)
# before,after =fun_Save_Duty_Cycle_Change_I('Z:\Thesis\Optislang_Motorcad\DutyCycleData','OP1_temp_rise',ipk)


# fun_Save_Duty_Cycle_Change_I(mot_file_dir,OP1_Duty_Cycle,ipk)



In [324]:
# 제약조건
o_OP1_max_temp,OP1_max_pos,OP1_temp_dic=fun_Check_Temp_Rise_allComponent(OP1)
o_OP2_max_temp,OP2_max_pos,OP2_temp_dic=fun_Check_Temp_Rise_allComponent(OP2)
o_OP3_max_temp,OP3_max_pos,OP3_temp_dic=fun_Check_Temp_Rise_allComponent(OP3)
print(o_OP1_max_temp,o_OP2_max_temp,o_OP3_max_temp,OP3_max_pos,OP2_max_pos)

OP3_temp_dic['Stator_Winding_Temp_Average'][1]

73.9151255402935 136.119759717194 186.411706974832 ['Airgap_Temp'] ['Stator_Winding_Temp_Average']


168.372190255515

Driving Duty Cycle Data Calc Time :
    M1

In [ ]:

### Lab: Energy Consumption over drive cycle

  # Settings
    # mcApp.SetVariable('N_d_MotorLAB', i_Gear_Ratio)                     # Gear Ratio

    mcApp.SetVariable("DutyCycleType_Lab", 0)                 # Automotive drive cycle
    # mcApp.SetVariable("DrivCycle_MotorLAB", "WLTP Class 3")   # WLTP3 drive cycle
    mcApp.SetVariable("LabThermalCoupling_DutyCycle", 0)      # No coupling with Thermal
    
  # Calculation & Post processing
    mcApp.CalculateDutyCycle_Lab()                                                # Run calculation
    #ex, o_WLTP3_Eff = mcApp.GetVariable("DutyCycleAverageEfficiency_EnergyUse")   # Get efficiency value 
    ex, o_Wh_Loss = mcApp.GetVariable("DutyCycleTotalLoss")   # Get efficiency value 
    ex, o_Wh_Shaft = mcApp.GetVariable("DutyCycleTotalEnergy_Shaft_Output")   # Get efficiency value 
    ex, o_Wh_input = mcApp.GetVariable("DutyCycleTotalEnergy_Electrical_Input")   # Get efficiency value 
      
  # Raise exception if wrong performance data
    if o_Wh_Loss < 0:
        mcApp.SaveToFile(mot_file_new_path)  # Save design   
        mcApp.Quit()                         # Close Motor-CAD
        mcApp = 0                            # Reset mcApp variable  
        time.sleep(0.5)                      # Frozen for 0.5s
        raise Exception('[ERROR] {}: drive cycle performance calculation failed'.format(OSL_DESIGN_NAME))


def output init

In [ ]:
def fun_
           o_Cont_Torque_1700rpm  = 0.  
        o_Cont_Torque_4krpm  = 0.

        o_Peak_Torque_1700rpm = 0.
        o_Peak_Power_1700rpm   = 0
        o_Wh_Loss           =0
        o_Wh_Shaft          =0
        o_Wh_input          =0
        o_Torque_Density     = 0.
        o_Op1_ipk            = 0.
        o_Weight_Act         = 0.
        o_Weight_Mag         = 0.
        o_Weight_Rot_Core    = 0.
        o_Weight_Stat_Core   = 0.
        o_Weight_Wdg         = 0.
        